In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import math
import random
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import datetime as dt
import os
np.random.seed(7)
from scipy.spatial import distance
from numpy.random import choice
import math
from scipy import stats
from scipy import optimize
import matplotlib as mpl
import seaborn as sns; sns.set()

Use this notebook to recreate the qlearning part, from optimization to plots.

In [2]:
mpl.use('svg')
new_rc_params = {
    "font.size": 20, #choosing the font size helps latex to place all the labels, ticks etc. in the right place
    "svg.fonttype": 'none',
    'figure.max_open_warning': 0
} 
mpl.rcParams.update(new_rc_params)
#     "font.weight":700
plt.rcParams['axes.facecolor'] = '0.95'

my_font_dict = {
        'size': 20,
        }

In [4]:
saving_dir = os.path.join(os.getcwd(), 'paper_additions')
if not os.path.exists(saving_dir):
    os.makedirs(saving_dir)
    
data = pd.read_csv('Data/DataAllSubjectsRewards.csv')
data.fillna(-1, inplace=True)

Load prior results

In [5]:
general_saving_dir = "cross_validation/averages/general/"
if not os.path.exists(general_saving_dir):
    os.makedirs(general_saving_dir)
    
reward_oriented_saving_dir = "cross_validation/averages/reward_oriented/"
if not os.path.exists(reward_oriented_saving_dir):
    os.makedirs(reward_oriented_saving_dir)
    
no_reward_saving_dir = "cross_validation/averages/no_reward/"
if not os.path.exists(no_reward_saving_dir):
    os.makedirs(no_reward_saving_dir)

In [6]:
# General
with open(os.path.join(general_saving_dir, 'general_all_folds_all_payoffs_true_false.pkl'), 'rb') as f:
    general_all_folds_all_payoffs_true_false = pickle.load(f)
general_all_folds_true_false, general_payoff2_persons_true_false_predictions_lstm, general_payoff3_persons_true_false_predictions_lstm, general_payoff4_persons_true_false_predictions_lstm = general_all_folds_all_payoffs_true_false

with open(os.path.join(general_saving_dir, 'general_all_accuracy_per_par.pkl'), 'rb') as f:
    general_all_accuracy_per_par = pickle.load(f)
general_all_folds_lstm_accs, general_payoff2_lstm_accs, general_payoff3_lstm_accs, general_payoff4_lstm_accs = general_all_accuracy_per_par


# Reward-Oblivious
with open(os.path.join(no_reward_saving_dir, 'no_reward_all_folds_all_payoffs_true_false.pkl'), 'rb') as f:
    no_reward_all_folds_all_payoffs_true_false = pickle.load(f)
no_reward_all_folds_true_false, no_reward_payoff2_persons_true_false_predictions_lstm, no_reward_payoff3_persons_true_false_predictions_lstm, no_reward_payoff4_persons_true_false_predictions_lstm = no_reward_all_folds_all_payoffs_true_false

with open(os.path.join(no_reward_saving_dir, 'no_reward_all_accuracy_per_par.pkl'), 'rb') as f:
    no_reward_all_accuracy_per_par = pickle.load(f)
no_reward_all_folds_lstm_accs, no_reward_payoff2_lstm_accs, no_reward_payoff3_lstm_accs, no_reward_payoff4_lstm_accs = no_reward_all_accuracy_per_par

FileNotFoundError: [Errno 2] No such file or directory: 'cross_validation/averages/general/general_all_folds_all_payoffs_true_false.pkl'

### Optimized qlearning

In [6]:
data_filled_NA = data
# data_noNA = full_orig_data.dropna()
# notice that here is where I choose if feeding full data with blanks, or without blanks
# without blanks:
# data_payoff2 = data_noNA[data_noNA['payoff_structure']==2]
# data_payoff3 = data_noNA[data_noNA['payoff_structure']==3]
# data_payoff4 = data_noNA[data_noNA['payoff_structure']==4]
# with blanks:
data_payoff2 = data_filled_NA[data_filled_NA['payoff_structure']==2]
data_payoff3 = data_filled_NA[data_filled_NA['payoff_structure']==3]
data_payoff4 = data_filled_NA[data_filled_NA['payoff_structure']==4]
user_ids2 = data_payoff2.user.unique()
user_ids3 = data_payoff3.user.unique()
user_ids4 = data_payoff4.user.unique()

In [7]:
def count_stickiness(person_df):
    """
    probability of switch - count how many times the person pressed the same choice again (first doesn't count)
    """
    sticky_count = 0
    for i,choice in enumerate(person_df.choice):
        if i==0:
            continue
        if (person_df.choice[i] == person_df.choice[i-1]):
            sticky_count+=1
    return sticky_count/i

In [13]:
def calc_mean_reward(person_df):
    return np.mean(person_df.reward)

In [8]:
# fixed version
# 30.10.21 paper addition - data needs to be full payoff
def q_learning_model(params, *args):
    choices = list(args[0].choice.astype(int))
    rewards = list(np.divide(list(args[0].reward.astype(int)),98))
    users = list(args[0].user.astype(int))
    num_of_trials = len(choices)

    prob_of_choice = [0]*num_of_trials
    accuracy = [0]*num_of_trials

    # notice: I want to init q to this for every new participant
    q = list(np.divide([50,50,50,50],98))
    init_q = list(np.divide([50,50,50,50],98))
    beta = params[0]
    alpha = params[1]


    q_predictions = []

    last_user = users[0]
    for trial in range(num_of_trials):
        if users[trial] != last_user:
            # print(f'in fitting, finished user {last_user} changing to next, init q values')
            last_user = users[trial]
            q = init_q
        if choices[trial] == -1:
            # if it's first trial, there's no previous prediction to count on - make a random choice then. Reward will be 0
            if trial == 0:
                random_q_ind = random.choice([0,1,2,3])
                prob_of_choice[trial] = np.divide( np.exp(np.multiply(beta, q[random_q_ind])),  np.sum(np.exp(np.multiply(beta,q)), axis=0))
                accuracy[trial] = int((choices[trial]-1)==random_q_ind)
                q_predictions.append(random_q_ind)
                # pe = 0 - q[random_q_ind]
                # q[random_q_ind] = q[random_q_ind] + alpha*pe
            else:
                prob_of_choice[trial] =  prob_of_choice[trial-1]
                accuracy[trial] = accuracy[trial]-1
                q_predictions.append(q_predictions[-1])
                # pe = 0 - q[q_predictions[-1]]
                # q[q_predictions[-1]] = q[q_predictions[-1]] + alpha*pe
        else:
            # regular choice case
            prob_of_choice[trial] = np.divide( np.exp(np.multiply(beta, q[choices[trial]-1])),  np.sum(np.exp(np.multiply(beta,q)), axis=0))
            accuracy[trial] = int((choices[trial]-1)==np.argmax(q))
            q_predictions.append(np.argmax(q))
            pe = rewards[trial]-q[choices[trial]-1]
            q[choices[trial]-1] = q[choices[trial]-1] + alpha*pe
        # prob_of_choice[trial] = np.divide( np.exp(np.multiply(beta, q[choices[trial]-1])),  np.sum(np.exp(np.multiply(beta,q)), axis=0))

        # pe = rewards[trial]-q[choices[trial]-1]
        # q[choices[trial]-1] = q[choices[trial]-1] + alpha*pe
        

    return -(np.sum(np.log(prob_of_choice)))

In [9]:
# fixed version
# 30.10.21 paper addition - data needs to be full payoff
def q_learning_model_fitted(params, data):
    beta = params.x[0]
    alpha = params.x[1]
    
    choices = list(data.choice.astype(int))
    rewards = list(np.divide(list(data.reward.astype(int)),98))

    num_of_trials = len(choices)
    prob_of_choice = [0]*num_of_trials
    accuracy = [0]*num_of_trials
    
    q = list(np.divide([50,50,50,50],98))

    q_predictions = []
    for trial in range(num_of_trials):
        if choices[trial] == -1:
            # if it's first trial, there's no previous prediction to count on - make a random choice then. Reward will be 0
            if trial == 0:
                random_q_ind = random.choice([0,1,2,3])
                prob_of_choice[trial] = np.divide( np.exp(np.multiply(beta, q[random_q_ind])),  np.sum(np.exp(np.multiply(beta,q)), axis=0))
                accuracy[trial] = int((choices[trial]-1)==random_q_ind)
                q_predictions.append(random_q_ind)
#                 pe = 0 - q[random_q_ind]
#                 q[random_q_ind] = q[random_q_ind] + alpha*pe
            else:
                prob_of_choice[trial] =  prob_of_choice[trial-1]
                accuracy[trial] = accuracy[trial]-1
                q_predictions.append(q_predictions[-1])
#                 pe = 0 - q[q_predictions[-1]]
#                 q[q_predictions[-1]] = q[q_predictions[-1]] + alpha*pe
        else:
            # regular choice case
            prob_of_choice[trial] = np.divide( np.exp(np.multiply(beta, q[choices[trial]-1])),  np.sum(np.exp(np.multiply(beta,q)), axis=0))
            accuracy[trial] = int((choices[trial]-1)==np.argmax(q))
            q_predictions.append(np.argmax(q))
            pe = rewards[trial]-q[choices[trial]-1]
            q[choices[trial]-1] = q[choices[trial]-1] + alpha*pe
        

    return -(np.sum(np.log(prob_of_choice))),accuracy, q_predictions

In [10]:
def get_qlearning_params(person_df, minimum):
    bounds = optimize.Bounds([0, 1], [0, 999])
    # fmin was ‘Nelder-Mead’ (downhill simplex)
    # minimum = optimize.fmin(q_learning_model,x0=(1,0.1),args=(person_df[['choice','reward']],))
    # minimize is using ‘L-BFGS-B’
    # minimum = optimize.minimize(q_learning_model,x0=(1,0.1),bounds=([0, np.inf], [0, 1]),args=(person_df[['choice','reward', 'user']],))
    ll, acc, q_predictions = q_learning_model_fitted(minimum, person_df)
    accuracy = np.divide(np.sum(acc),len(acc))
    return ll,acc,accuracy, q_predictions

In [11]:
def get_real_accuracy(person_df, payoff_person_predictions):
    person_choices = person_df.choice -1
    correct = 0
    count = 0
    for choice,pred in zip(person_choices,payoff_person_predictions[-1]):
        if choice == pred:
            correct+=1
        if choice>=0:
            count+=1
    return (correct/count)

In [121]:
data_payoff2.head()

,user,choice,reward,time,payoff_structure,reward_1,reward_2,reward_3,reward_4
0,1,1.0,84.0,1104.0,2,84,87,42,23
1,1,2.0,90.0,1076.0,2,90,90,46,18
2,1,3.0,53.0,612.0,2,80,84,53,28
3,1,4.0,24.0,742.0,2,87,81,50,24
4,1,2.0,92.0,927.0,2,86,92,61,28


In [20]:
# instead of running each one seperatly, do one run for all - to get one alpha and beta
############ PAYOFF2 #############################################

# ll, acc,accuracy, q_predictions = get_qlearning_params(data_payoff2, minimum)
#  acc_per_user = get_real_accuracy(data_payoff2)

# updated: doing optimization only once, for all
minimum = optimize.minimize(q_learning_model,x0=(1,0.1),bounds=([0, np.inf], [0, 1]),args=(data_filled_NA[['choice','reward', 'user']],))


print("starting payoff2")
############ PAYOFF2 #############################################
payoff_2_person_features = []
payoff_2_person_accuracies = []    # out of 150
payoff_2_person_predictions = []
payoff_2_person_real_accuracies = []    # out of choices number

# minimum = optimize.minimize(q_learning_model,x0=(1,0.1),bounds=([0, np.inf], [0, 1]),args=(data_payoff2[['choice','reward', 'user']],))

for participant in user_ids2:
    person_df = data_payoff2[data_payoff2['user']==participant].reset_index(drop=True).copy()
    ft_1 = count_stickiness(person_df)
    ft_2 = calc_mean_reward(person_df)
    ft_3 = minimum.x[0] # beta
    ft_4 = minimum.x[1] # alpha
    ft_5,acc,accuracy, q_predictions = get_qlearning_params(person_df, minimum)
    payoff_2_person_features.append([ft_1,ft_2,ft_3,ft_4,ft_5])
    payoff_2_person_accuracies.append(accuracy)
    payoff_2_person_predictions.append(q_predictions)

    payoff_2_person_real_accuracies.append(get_real_accuracy(person_df, payoff_2_person_predictions))
##################################################################
print("starting payoff3")
############ PAYOFF3 #############################################
payoff_3_person_features = []
payoff_3_person_accuracies = []    # out of 150
payoff_3_person_predictions = []
payoff_3_person_real_accuracies = []    # out of choices number

# minimum = optimize.minimize(q_learning_model,x0=(1,0.1),bounds=([0, np.inf], [0, 1]),args=(data_payoff3[['choice','reward', 'user']],))

for participant in user_ids3:
    person_df = data_payoff3[data_payoff3['user']==participant].reset_index(drop=True).copy()
    ft_1 = count_stickiness(person_df)
    ft_2 = calc_mean_reward(person_df)
    ft_3 = minimum.x[0]
    ft_4 = minimum.x[1]
    ft_5,acc,accuracy, q_predictions = get_qlearning_params(person_df, minimum)
    payoff_3_person_features.append([ft_1,ft_2,ft_3,ft_4,ft_5])
    payoff_3_person_accuracies.append(accuracy)
    payoff_3_person_predictions.append(q_predictions)

    payoff_3_person_real_accuracies.append(get_real_accuracy(person_df, payoff_3_person_predictions))
##################################################################


##################################################################
print("starting payoff4")
############ PAYOFF4 #############################################
payoff_4_person_features = []
payoff_4_person_accuracies = []  # out of 150
payoff_4_person_predictions = []
payoff_4_person_real_accuracies = []  # out of choices number

# minimum = optimize.minimize(q_learning_model, x0=(1, 0.1), bounds=([0, np.inf], [0, 1]), args=(data_payoff4[['choice', 'reward', 'user']],))

for participant in user_ids4:
    person_df = data_payoff4[data_payoff4['user'] == participant].reset_index(drop=True).copy()
    ft_1 = count_stickiness(person_df)
    ft_2 = calc_mean_reward(person_df)
    ft_3 = minimum.x[0]
    ft_4 = minimum.x[1]
    ft_5, acc, accuracy, q_predictions = get_qlearning_params(person_df, minimum)
    payoff_4_person_features.append([ft_1, ft_2, ft_3, ft_4, ft_5])
    payoff_4_person_accuracies.append(accuracy)
    payoff_4_person_predictions.append(q_predictions)

    payoff_4_person_real_accuracies.append(get_real_accuracy(person_df, payoff_4_person_predictions))
##################################################################


starting payoff2
starting payoff3
starting payoff4


Create true_false_prediction_mat: a indicates if a prediction is correct or wrong, per user per choice. Later is being 

In [12]:
def get_true_false_predictions(person_df, participant, payoff_qlearn_person_predictions, payoff_person_real_accuracies, payoff_length_correction):
    """
    gets the person_df and the curresponding user id (participant) with the qlearning predictions (and payoff_person_real_accuracies for assertion), and returns a true/false vector of 150
    """
    person_choices = person_df.choice -1
    participant_true_false = []
    for choice, prediction in zip(person_choices, payoff_qlearn_person_predictions[participant-1-payoff_length_correction]):
        if choice<0:
            participant_true_false.append(np.nan)
        else:
            if choice == prediction:
                participant_true_false.append(1)
            else:
                participant_true_false.append(0)

    participant_true_false_np = np.asarray(participant_true_false)
    assert(np.nanmean(participant_true_false_np)==payoff_person_real_accuracies[participant-1-payoff_length_correction]),"accuracy is wrong"
    participant_true_false_np = np.nan_to_num(participant_true_false_np, -1)
    return participant_true_false_np

In [22]:
## QLEARNING MODEL
## for each paticipant, get a vector of 150 predictions where each cell is (0 if false, 1 if correct, -1 if blank)
############ PAYOFF2 #############################################
payoff_2_person_true_false_predictions = []
for participant in user_ids2:
    person_df = data_payoff2[data_payoff2['user']==participant].reset_index(drop=True).copy()
    payoff_2_person_true_false_predictions.append(get_true_false_predictions(person_df, participant, payoff_2_person_predictions, payoff_2_person_real_accuracies, payoff_length_correction=0))

payoff_2_person_true_false_predictions_matrix_qlearning = np.asmatrix(payoff_2_person_true_false_predictions)
##################################################################


############ PAYOFF3 #############################################
payoff_3_person_true_false_predictions = []
for participant in user_ids3:
    person_df = data_payoff3[data_payoff3['user']==participant].reset_index(drop=True).copy()
    payoff_3_person_true_false_predictions.append(get_true_false_predictions(person_df, participant, payoff_3_person_predictions, payoff_3_person_real_accuracies, payoff_length_correction=len(user_ids2)))

payoff_3_person_true_false_predictions_matrix_qlearning = np.asmatrix(payoff_3_person_true_false_predictions)
##################################################################


############ PAYOFF4 #############################################
payoff_4_person_true_false_predictions = []
for participant in user_ids4:
    person_df = data_payoff4[data_payoff4['user']==participant].reset_index(drop=True).copy()
    payoff_4_person_true_false_predictions.append(get_true_false_predictions(person_df, participant, payoff_4_person_predictions, payoff_4_person_real_accuracies, payoff_length_correction=len(user_ids2)+len(user_ids3)))

payoff_4_person_true_false_predictions_matrix_qlearning = np.asmatrix(payoff_4_person_true_false_predictions)
##################################################################

In [23]:
payoff2_qlearn_plots_res = payoff_2_person_features, payoff_2_person_accuracies, payoff_2_person_real_accuracies, payoff_2_person_predictions, payoff_2_person_true_false_predictions_matrix_qlearning
payoff3_qlearn_plots_res = payoff_3_person_features, payoff_3_person_accuracies, payoff_3_person_real_accuracies, payoff_3_person_predictions, payoff_3_person_true_false_predictions_matrix_qlearning
payoff4_qlearn_plots_res = payoff_4_person_features, payoff_4_person_accuracies, payoff_4_person_real_accuracies, payoff_4_person_predictions, payoff_4_person_true_false_predictions_matrix_qlearning

In [25]:
# Updated: saved on 1.11.21
with open(os.path.join(saving_dir,'payoff2_qlearn_reproduce_fixed_paper.pkl'), 'wb') as handle:
    pickle.dump(payoff2_qlearn_plots_res, handle)


with open(os.path.join(saving_dir,'payoff3_qlearn_reproduce_fixed_paper.pkl'), 'wb') as handle:
    pickle.dump(payoff3_qlearn_plots_res, handle)

with open(os.path.join(saving_dir,'payoff4_qlearn_reproduce_fixed_paper.pkl'), 'wb') as handle:
    pickle.dump(payoff4_qlearn_plots_res, handle)

In [14]:
# load for debugging
import os
saving_dir = '/Users/matanfintz/workspace/thesis_code/Thesis/paper_additions/'
with open(os.path.join(saving_dir,'payoff2_qlearn_reproduce_fixed_paper.pkl'), 'rb') as handle:
    payoff2_qlearn_plots_res = pickle.load(handle)


with open(os.path.join(saving_dir,'payoff3_qlearn_reproduce_fixed_paper.pkl'), 'rb') as handle:
    payoff3_qlearn_plots_res = pickle.load(handle)

with open(os.path.join(saving_dir,'payoff4_qlearn_reproduce_fixed_paper.pkl'), 'rb') as handle:
    payoff4_qlearn_plots_res = pickle.load(handle)


payoff_2_person_features, payoff_2_person_accuracies, payoff_2_person_real_accuracies, payoff_2_person_predictions, payoff_2_person_true_false_predictions_matrix_qlearning = payoff2_qlearn_plots_res

payoff_3_person_features, payoff_3_person_accuracies, payoff_3_person_real_accuracies, payoff_3_person_predictions, payoff_3_person_true_false_predictions_matrix_qlearning = payoff3_qlearn_plots_res

payoff_4_person_features, payoff_4_person_accuracies, payoff_4_person_real_accuracies, payoff_4_person_predictions, payoff_4_person_true_false_predictions_matrix_qlearning = payoff4_qlearn_plots_res

In [28]:
# loading_dir = 'explain/qlearning_model_plots/'
# with open(os.path.join(loading_dir,'payoff2_qlearn_reproduce_tf.pkl'), 'rb') as handle:
#     payoff2_qlearn_plots_res = pickle.load(handle)

    
# with open(os.path.join(loading_dir,'payoff3_qlearn_reproduce_tf.pkl'), 'rb') as handle:
#     payoff3_qlearn_plots_res = pickle.load(handle)

# with open(os.path.join(loading_dir,'payoff4_qlearn_reproduce_tf.pkl'), 'rb') as handle:
#     payoff4_qlearn_plots_res = pickle.load(handle)

    
payoff_2_person_features, payoff_2_person_accuracies, payoff_2_person_real_accuracies, payoff_2_person_predictions, payoff_2_person_true_false_predictions_matrix_qlearning = payoff2_qlearn_plots_res
payoff_3_person_features, payoff_3_person_accuracies, payoff_3_person_real_accuracies, payoff_3_person_predictions, payoff_3_person_true_false_predictions_matrix_qlearning = payoff3_qlearn_plots_res 
payoff_4_person_features, payoff_4_person_accuracies, payoff_4_person_real_accuracies, payoff_4_person_predictions, payoff_4_person_true_false_predictions_matrix_qlearning = payoff4_qlearn_plots_res 

In [29]:
def get_avg_acc_per_time(payoff_person_true_false_predictions_matrix):
    """
    gets a payoff_person_true_false_predictions_matrix -  which can be either qlearning or lstm 
    return a vector of 150, each entrance is the avg accuracy across all participants in that payoff
    """
    payoff_all_pars_avg_acc_per_time = []
    for t in range(payoff_person_true_false_predictions_matrix.shape[1]):
        collect_not_minuses = []
        for pred in payoff_person_true_false_predictions_matrix[:,t]:
            if pred<0:
                continue
            else:
                collect_not_minuses.append(pred)
        payoff_all_pars_avg_acc_per_time.append(np.mean(collect_not_minuses))
    
    return payoff_all_pars_avg_acc_per_time

In [30]:
def plot_accuracy_over_time(true_false_predictions_matrix_qlearning, true_false_predictions_matrix_lstm, payoff, legend_1='Qlearning acc', legend_2='LSTM acc', save_fig=False, add_text=""):
    """
    Ploting accuracy comparison over time between 2 models
    """
    plt.clf()
    fig = plt.gcf()
    fig.set_size_inches(15, 5)
    x = np.arange(4,150)
    plt.plot(x, get_avg_acc_per_time(true_false_predictions_matrix_qlearning)[4:], 'r') 
    plt.plot(x, get_avg_acc_per_time(true_false_predictions_matrix_lstm)[4:], 'b') 
    plt.legend([legend_1, legend_2])
    plt.title('Payoff{} accuracy over time Qlearn vs LSTM'.format(payoff), fontdict=my_font_dict)
    plt.xlabel('Time', fontdict=my_font_dict)
    plt.ylabel('Accuracy', fontdict=my_font_dict)
    plt.tick_params(axis='both', labelsize=20)
    if save_fig:
        fig.savefig(os.path.join(saving_dir,'payoff{}_accuracy_over_time_{}_Qlearn_vs_LSTM.svg'.format(payoff, add_text)), dpi=200, format='svg')
    plt.show()

In [31]:
def plot_accuracy_over_time_all_together(true_false_pred_list:list, payoff:int, legend:list, save_fig=False, add_text=""):
    """
    Ploting accuracy comparison over time between (2) models
    """
    plt.clf()
    fig = plt.gcf()
    fig.set_size_inches(22, 8)
    x = np.arange(4,150)
    colors = ['r','g','b']
    for pred_mat,color in zip(true_false_pred_list, colors):
        plt.plot(x, get_avg_acc_per_time(pred_mat)[4:], color) 
    
    plt.legend([l for l in legend])
    plt.title('payoff{}_accuracy_over_time_Qlearn_vs_LSTM'.format(payoff))
    plt.xlabel('time')
    plt.ylabel('accuracy')
    if save_fig:
        fig.savefig(os.path.join(saving_dir,'payoff{}_accuracy_over_time_{}_Qlearn_vs_LSTM.svg'.format(payoff, add_text)), dpi=200, format='svg')
    plt.show()

In [32]:
# # divide General lstm true/false matrix to payoffs
# payoff2_persons_true_false_predictions_lstm = np.asmatrix(persons_true_false_predictions[0:329])
# payoff3_persons_true_false_predictions_lstm = np.asmatrix(persons_true_false_predictions[329:641])
# payoff4_persons_true_false_predictions_lstm = np.asmatrix(persons_true_false_predictions[641:965])

In [33]:
plot_accuracy_over_time(payoff_2_person_true_false_predictions_matrix_qlearning, general_payoff2_persons_true_false_predictions_lstm, payoff=2, save_fig=True)

/Users/matanfintz/opt/anaconda3/envs/ident/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/matanfintz/opt/anaconda3/envs/ident/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-30-9916d8112409>:18: UserWarning: Matplotlib is currently using svg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [34]:
plot_accuracy_over_time(payoff_3_person_true_false_predictions_matrix_qlearning, general_payoff3_persons_true_false_predictions_lstm, payoff=3, save_fig=True)

/Users/matanfintz/opt/anaconda3/envs/ident/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/matanfintz/opt/anaconda3/envs/ident/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-30-9916d8112409>:18: UserWarning: Matplotlib is currently using svg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [35]:
plot_accuracy_over_time(payoff_4_person_true_false_predictions_matrix_qlearning, general_payoff4_persons_true_false_predictions_lstm, payoff=4, save_fig=True)

/Users/matanfintz/opt/anaconda3/envs/ident/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/matanfintz/opt/anaconda3/envs/ident/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-30-9916d8112409>:18: UserWarning: Matplotlib is currently using svg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [36]:
def extract_lstm_acc_per_person(persons_true_false_predictions):
    collect_not_minuses = []
    for pred in persons_true_false_predictions:
        if pred<0:
            continue
        else:
            collect_not_minuses.append(pred)
    return np.mean(collect_not_minuses)
#     return np.mean(persons_true_false_predictions[4:])

In [37]:
payoff2_lstm_accs = []
for user_ind in user_ids2:
    payoff2_lstm_accs.append(extract_lstm_acc_per_person(general_all_folds_true_false[user_ind-1]))
    
    
payoff3_lstm_accs = []
for user_ind in user_ids3:
    payoff3_lstm_accs.append(extract_lstm_acc_per_person(general_all_folds_true_false[user_ind-1]))
    
    
payoff4_lstm_accs = []
for user_ind in user_ids4:
    payoff4_lstm_accs.append(extract_lstm_acc_per_person(general_all_folds_true_false[user_ind-1]))

In [38]:
def create_scatter_all_payoffs(all_dfs, x_axis, y_axis, add_text='', save_fig=False, save_format='svg'):
    """
    scater of participants accuracy of all 3 payoffs with 2 model types. 
    notice that the order of the dfs passed HAS to be the same order as 2,3,4 .
    """
    plt.clf()
    sns.set(rc={'figure.figsize':(12,12)})
    comparison_title = '{} VS {}'.format(x_axis, y_axis, fontdict=my_font_dict)
    
    
    ax = sns.scatterplot(x=x_axis, y=y_axis, data=all_dfs,hue='Payoff',legend='full', palette=['r','g','b'])
    
    plt.title("All payoffs participants accuracy comparison {} {}".format(comparison_title, add_text), fontdict=my_font_dict)
    
    x1, y1 = [0, 1], [0, 1]
#     plt.legend(['payoff_2','payoff_3','payoff_4'])
    plt.plot(x1, y1,'k--', marker = 'x')
    plt.tick_params(axis='both', labelsize=20)
    if save_fig:
        if save_format=='svg':
            plt.savefig(os.path.join(saving_dir,'payoff_participant_accuracy_comparison_{}.svg'.format(comparison_title.replace(" ","_"))),quality=95,dpi=200, format='svg')
        else:
            plt.savefig(os.path.join(saving_dir,'payoff_participant_accuracy_comparison_{}.png'.format(comparison_title.replace(" ","_"))),quality=95,dpi=200)
    plt.show()
#     mpl.get_backend()

In [39]:
gen_vs_ql_p2_acc_df = pd.DataFrame({'Qlearning model':payoff_2_person_accuracies,'LSTM':general_payoff2_lstm_accs})
gen_vs_ql_p3_acc_df = pd.DataFrame({'Qlearning model':payoff_3_person_accuracies,'LSTM':general_payoff3_lstm_accs})
gen_vs_ql_p4_acc_df = pd.DataFrame({'Qlearning model':payoff_4_person_accuracies,'LSTM':general_payoff4_lstm_accs})

In [40]:
# combine all to one df
_data=[]
for row in gen_vs_ql_p2_acc_df.itertuples():
    _data.append({'Qlearning model':row[1],'LSTM':row[2],'Payoff':2})
df2 =  pd.DataFrame(_data)

_data=[]
for row in gen_vs_ql_p3_acc_df.itertuples():
    _data.append({'Qlearning model':row[1],'LSTM':row[2],'Payoff':3})
df3 =  pd.DataFrame(_data)

_data=[]
for row in gen_vs_ql_p4_acc_df.itertuples():
    _data.append({'Qlearning model':row[1],'LSTM':row[2],'Payoff':4})
df4 =  pd.DataFrame(_data)

gen_vs_ql_all_dfs= pd.concat([df2,df3,df4])

In [42]:
# create_scatter_all_payoffs(dfs=[gen_vs_ql_p2_acc_df,gen_vs_ql_p3_acc_df,gen_vs_ql_p4_acc_df], x_axis='Qlearning_Model', y_axis='LSTM' ,save_fig=True)
create_scatter_all_payoffs(gen_vs_ql_all_dfs,x_axis='Qlearning model', y_axis='LSTM' ,save_fig=True)

<ipython-input-38-56187883ff6d>:22: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "quality" which is no longer supported as of 3.3 and will become an error two minor releases later
  plt.savefig(os.path.join(saving_dir,'payoff_participant_accuracy_comparison_{}.svg'.format(comparison_title.replace(" ","_"))),quality=95,dpi=200, format='svg')
<ipython-input-38-56187883ff6d>:25: UserWarning: Matplotlib is currently using svg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [15]:
# summary results
# pd.DataFrame({'payoff2':[np.mean(payoff_2_person_real_accuracies)],
#               'payoff3':[np.mean(payoff_3_person_real_accuracies)],
#               'payoff4':[np.mean(payoff_4_person_real_accuracies)],
#               'full_avg':q_leanr_avg
#               },
#              index=['q_learn_model'])